# Ruteo

## Preparación de la Sesión

In [1]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [2]:
cd ope3-logistica

/content/ope3-logistica


In [3]:
!pip install -e . &> /dev/null

## Módulos

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

In [5]:
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

In [6]:
from logistica import metaheuristicas as mh

# Carga de Datos

In [7]:
df_clientes = load_inputs("clientes")
df_pedidos = load_inputs("pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [8]:
df_camiones = load_inputs("camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


# Optimización 1 Pedido


In [136]:
#@title Seleccionar Pedido
pedido = 'pedidos3' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [137]:
df_pedido = preparar_df_pedidos(df_pedidos, pedido)
df_pedido

,cliente,pedidos,coord_x,coord_y
0,A,5,0.257571,1.803726
1,B,5,1.523313,2.102301
2,C,6,0.710910,2.629800
3,D,5,1.013940,2.149740
4,E,2,0.609480,0.666810
5,F,6,0.998640,1.451790
6,G,3,1.434510,0.018180
7,H,5,2.171880,0.764910
8,I,3,1.412280,0.863190
9,J,6,2.386260,1.387260


In [138]:
#@title Parámetros del Ruteo
costo_oportunidad = 3000 #@param {type:"number"}
presupuesto = 1220 #@param {type:"number"}

In [139]:
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=costo_oportunidad, presupuesto=presupuesto)

In [140]:
#@title Parámetros 
mode = 'random' #@param ["simple", "random"]
random_state = 45 #@param {type:"number"}

In [141]:
ruteo.get_solucion_inicial(mode=mode, random_state=random_state)
ruteo

--Ruteo--
Carga Total: 55tn
Costo Camiones: 61600$
Costo Oportunidad: 0$ 
Costo Total: 61600$ 
Costo Total por tn: 1120.0$/tn 
Ahorro: -8.2%

In [142]:
#@title Parámetros Metaheurística
temp_inicial = 50 #@param {type:"number"}
temp_final = 1 #@param {type:"number"}
k = 100 #@param {type:"number"}
iters = 10 #@param {type:"number"}
temp_mode = 'non_linear' #@param ["linear", "non_linear"]
p = 1 #@param {type:"number"}
random_state = 42 #@param {type:"number"}

In [143]:
best_sol, history = mh.sa(ruteo, 
                          t_inicial=temp_inicial, 
                          t_final=temp_final, 
                          k=k, 
                          iters=iters, 
                          temp_mode=temp_mode, 
                          prob=p, 
                          random_state=random_state)
print("\n")
print(best_sol)
print(f'Tiempo Optimización: {round(history.get("time"),2)} segundos')
print(f'Iteraciones Totales: {len(history.get("new_sol"))}')

100%|██████████| 100/100 [00:00<00:00, 135.43it/s]



--Ruteo--
Carga Total: 55tn
Costo Camiones: 59000$
Costo Oportunidad: 0$ 
Costo Total: 59000$ 
Costo Total por tn: 1072.73$/tn 
Ahorro: -12.07%
Tiempo Optimización: 0.75 segundos
Iteraciones Totales: 1000


In [144]:
best_sol.summary_camiones()

,Cliente 1,Cliente 2,Cliente 3,Costo,Carga,Costo_tn,Ahorro
Camion 1,L,None,None,5600,3,1866.67,53.01
Camion 2,H,K,None,11000,11,1000.0,-18.03
Camion 3,D,F,None,11000,11,1000.0,-18.03
Camion 4,E,B,None,8400,7,1200.0,-1.64
Camion 5,G,J,I,12000,12,1000.0,-18.03
Camion 6,C,A,None,11000,11,1000.0,-18.03
Costo Oportunidad,,,,0,0,,
Total,,,,59000,55,1072.73,-12.07


In [145]:
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,5,True,6
Pedido B,5,True,4
Pedido C,6,True,6
Pedido D,5,True,3
Pedido E,2,True,4
Pedido F,6,True,3
Pedido G,3,True,5
Pedido H,5,True,2
Pedido I,3,True,5
Pedido J,6,True,5


In [146]:
df_history = pd.DataFrame({"temp":history["temp"],
                           "actual_sol":history["actual_sol"],
                           "new_sol":history["new_sol"]})

df_history = (df_history
              .assign(best_sol = lambda df_: df_.actual_sol.cummin(),
                      delta = lambda df_: df_.actual_sol - df_.new_sol,
                      p = lambda df_: np.exp(df_.delta/df_.temp))
              .assign(p = lambda df_: np.round(np.where(df_.p > 1, 1, df_.p) ,2)))

df_history

,temp,actual_sol,new_sol,best_sol,delta,p
0,49.924941,1120.00,1120.00,1120.00,0.00,1.00
1,49.924941,1120.00,1120.00,1120.00,0.00,1.00
2,49.924941,1120.00,1120.00,1120.00,0.00,1.00
3,49.924941,1120.00,1130.91,1120.00,-10.91,0.80
4,49.924941,1130.91,1138.18,1120.00,-7.27,0.86
...,...,...,...,...,...,...
995,0.045553,1072.73,1083.64,1072.73,-10.91,0.00
996,0.045553,1072.73,1101.82,1072.73,-29.09,0.00
997,0.045553,1072.73,1090.91,1072.73,-18.18,0.00
998,0.045553,1072.73,1083.64,1072.73,-10.91,0.00


In [147]:
px.line(df_history,
        x=df_history.index,
        y=["new_sol", "actual_sol"],
        width=1000,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y=["delta", "p"],
        width=1000,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y="temp",
        width=1000,
        template="plotly_white").show()

px.line(df_history,
        x=df_history.index,
        y="best_sol",
        width=1000,
        template="plotly_white").show()

In [148]:
from plotly.subplots import make_subplots

In [149]:
fig1 = px.line(df_history,
               x=df_history.index,
               y=["new_sol", "actual_sol"],
               color_discrete_map={"new_sol":"#4e68c7", "actual_sol":"#db8344"})

fig2 = px.line(df_history,
               x=df_history.index,
               y=["delta", "p"],
               color_discrete_map={"delta":"#4e68c7", "p":"#d43a22"})

fig3 = px.line(df_history,
               x=df_history.index,
               y=["temp"],
               color_discrete_map={"temp":"#4e68c7"})

fig4 = px.line(df_history,
               x=df_history.index,
               y=["best_sol"],
               color_discrete_map={"best_sol":"#22a7d4"})

In [150]:
fig = make_subplots(rows=2, cols=2,
                    column_widths=[0.5, 0.5],
                    row_heights=[0.5, 0.5],
                    subplot_titles=['Actual y Nueva Solución', 
                                    'Temperatura', 
                                    'Delta y Probabilidad de Cambio', 
                                    'Mejor Solución'],
                    shared_xaxes=True)

traces = []
for i, figure in enumerate([fig1, fig2, fig3, fig4]):
  traces.append([])
  for trace in range(len(figure["data"])):
    traces[i].append(figure["data"][trace])

ubicacion = {0:[1,1], 1:[2,1], 2:[1,2], 3:[2,2]}

for i in range(4):
  for trace in traces[i]:
    fig.append_trace(trace, row=ubicacion[i][0], col=ubicacion[i][1])

fig.update_layout(template="plotly_white",
                  height=800,
                  width=1600,
                  legend=dict(orientation="h",
                              yanchor="bottom",
                              y=1.08,
                              xanchor="center",
                              x=0.5))

fig.show()

In [ ]:
# FALTA EL GRÁFICO DE LOS PEDIDOS EN X Y